# Jiangning WS

In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc

In [2]:
dtypes = {
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'hour'          : 'uint8',
        'is_attributed' : 'uint8', 
        'ip_day_hour_count': 'uint32', 
        'ip_os_day_hour_count': 'uint32', 
        'ip_app_day_hour_count': 'uint32', 
        'ip_app_os_day_hour_count': 'uint32', 
        'app_day_hour_count': 'uint32', 
        'ip_device_os_count': 'uint32', 
        'ip_app_device_os_count': 'uint32', 
        'ip_device_os_mean': 'float32',
        'ip_app_device_os_mean': 'float32',
        'ip_app_device_mean': 'float32',
        'app_device_os_mean': 'float32',
        'ip_device_os_time2nextclick': 'int32',
        'ip_app_device_os_time2nextclick': 'int32',
        'ip_app_device_time2nextclick': 'int32',
        'ip_device_os_time2previousclick': 'int32',
        'ip_app_device_os_time2previousclick': 'int32',
        'ip_app_device_time2previousclick': 'int32',
        'ip_device_os_countfromfuture': 'uint32', 
        'ip_app_device_os_countfromfuture': 'uint32', 
        'ip_app_device_countfromfuture': 'uint32', 
        'ip_device_os_countfrompast': 'uint32', 
        'ip_app_device_os_countfrompast': 'uint32', 
        'ip_app_device_countfrompast': 'uint32', 
        'ip_device_os_lasttimediff': 'int32',
        'ip_app_device_os_lasttimediff': 'int32',
        'ip_app_device_lasttimediff': 'int32',
        'ip_device_os_firsttimediff': 'int32',
        'ip_app_device_os_firsttimediff': 'int32',
        'ip_app_device_firsttimediff': 'int32',
        'matrixFact_user_iposdeviceapp_item_app': 'float32',
        'matrixFact_user_ip_item_appdeviceos': 'float32',
        'matrixFact_user_ipchannel_item_appdeviceos': 'float32',
        'ip_device_os_regression': 'float32',
        'ip_app_device_os_regression': 'float32',
        'ip_app_device_regression': 'float32',
        'ip_app_device_os_channel_regression': 'float32', 
        'attributed_timediffmax':'int32',
        'attributed_timediffmin':'int32',
        'attributed_timediff':'float32',
        'matrixFact_user_ipappdeviceos_item_channel': 'float32'
    
        } 

In [3]:
load_path = '/home/kai/talkingdata/data/jchen/'
# file_format = '{}_features_supplementV1.ftr'
file_format = '{}compressed_features_supplementV1.ftr'

day_list = ['day7', 'day8', 'day9']
df_dict = {}
for file in ['day7', 'day8', 'day9', 'test']: 
    df_dict[file] = pd.read_feather(load_path+file_format.format(file))
#     df_extra[file] = pd.read_csv(load_path+'{}_features_supplementV2_extra_timediff.csv'.format(file),dtype=dtypes)
    print(file_format.format(file))
# df_dict['test'] = pd.read_feather(load_path+file_format.format('test'))
# df_extra['test'] =  pd.read_csv(load_path+'test_equalhoursV2_features_supplementV2_extra_timediff.csv',dtype=dtypes)

    

day7compressed_features_supplementV1.ftr
day8compressed_features_supplementV1.ftr
day9compressed_features_supplementV1.ftr
testcompressed_features_supplementV1.ftr


In [4]:
# for day in  ['day7', 'day8', 'day9', ]:
#     print(day)
#     for col in df_dict['day7'].columns:
#         if col in dtypes.keys():
#             df_dict[day][col] = df_dict[day][col].astype(dtypes[col])
# print('test')
# for col in df_dict['test'].columns:
#     if col in dtypes.keys():
#         df_dict['test'][col] = df_dict['test'][col].astype(dtypes[col])
            
# #### saving

# for day in  ['day7', 'day8', 'day9', 'test']:
#     file_name = load_path + file_format.format(day+'compressed')
#     print(file_name)
#     df_dict[day].to_feather(file_name)

day7
day8
day9
test
/home/kai/talkingdata/data/jchen/day7compressed_features_supplementV1.ftr
/home/kai/talkingdata/data/jchen/day8compressed_features_supplementV1.ftr
/home/kai/talkingdata/data/jchen/day9compressed_features_supplementV1.ftr
/home/kai/talkingdata/data/jchen/testcompressed_features_supplementV1.ftr


# define parameter

In [4]:
target = 'is_attributed'

day_list = ['day7', 'day8', 'day9']

combine = 0
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 4000,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 32, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':60,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']
feature_cols = list(df_dict['test'].columns.values)

In [ ]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    if best_round is not None:
        param['num_rounds'] = best_round
        del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model

In [ ]:
# for day in day_list:
for day in ['day9']:
# for day in ['day7', 'day8']:
    train_day = list(set(day_list)-set([day]))
    file_name = 'concat_{}{}_val_{}_allday_supplement_V2_attributediff_noreg.npy'.format(train_day[0],train_day[1],day)
    print(file_name)
    trainset = pd.concat([df_dict[train_day[0]],df_dict[train_day[1]]])
    valset = df_dict[day]
    print('building train val done!')
    
    model = train_lightgbm(trainset, valset, feature_cols, categorical_col, params)
    best_round = model.best_iteration
#     best_round=163
    
    df_all = pd.concat([trainset, valset])
    del trainset

    gc.collect()
    model = train_lightgbm(df_all, valset, feature_cols, categorical_col, params, best_round)
    del df_all
    gc.collect()

    pred = model.predict(df_dict['test'][feature_cols])
    np.save(load_path+file_name, pred)

    # prediction
    df_test_raw = pd.read_csv('/home/kai/talkingdata/data/test.csv')
    print('loading file done!')
    df_sub = pd.DataFrame()
    df_sub['click_id'] = df_test_raw['click_id']
    df_sub['is_attributed'] = pred
    print('predicting file done!')
    df_sub.to_csv('/home/kai/talkingdata/data/jchen/submission/concat_{}{}_val_{}_allday_supplement_V2_bestround_{}_attributediff_noreg.csv.gz'.format(train_day[0],train_day[1],day,best_round), compression='gzip', index=False)

    

concat_day7day8_val_day9_allday_supplement_V2_attributediff_noreg.npy
building train val done!
start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 60 rounds.
[10]	valid_0's auc: 0.977532
[20]	valid_0's auc: 0.979521
[30]	valid_0's auc: 0.980683


In [ ]:
# appe = pd.read_csv('/home/kai/talkingdata/data/jchen/submission/concat_day8day7_val_day9_allday_supplement_V2_bestround_163_attributediff_noreg.csv.gz')
# zindex = np.load('/home/kai/talkingdata/data/idx_to_0.npy')
# appe.loc[zindex, 'is_attributed'] = 0
# print('start saving')
# appe.to_csv('/home/kai/talkingdata/data/jchen/submission/zeroconcat_day8day7_val_day9_allday_supplement_V2_bestround_163_attributediff_noreg.csv.gz',compression='gzip', index=False)